# StyleSense - AI Fashion Recommendation System
### Fixed Version - MediaPipe API Updated!

In [ ]:
# Cell 1: Install Required Packages
!pip install transformers diffusers accelerate safetensors opencv-python mediapipe gradio -q

In [ ]:
# Cell 2: Import Libraries
import torch
from diffusers import StableDiffusionPipeline
import gradio as gr
import cv2
import numpy as np
from PIL import Image

print("✓ All libraries imported successfully!")

In [ ]:
# Cell 3: Initialize Stable Diffusion Pipeline
print("Loading Stable Diffusion model... (this may take 3-5 minutes)")

model_id = "runwayml/stable-diffusion-v1-5"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    safety_checker=None,
    requires_safety_checker=False
)

device = "cuda" if torch.cuda.is_available() else "cpu"
pipe = pipe.to(device)

# Memory optimizations for Colab GPU
if device == "cuda":
    pipe.enable_attention_slicing()
    pipe.enable_vae_slicing()

print(f"✓ Model loaded successfully on {device.upper()}!")
print(f"✓ Ready to generate fashion images!")

In [ ]:
# Cell 4: Initialize MediaPipe for Skin Tone Detection (FIXED!)
# Using the new MediaPipe API
try:
    # Try new API first (MediaPipe 0.10+)
    from mediapipe.tasks import python
    from mediapipe.tasks.python import vision
    USE_NEW_API = True
    print("✓ Using MediaPipe new API (0.10+)")
except:
    # Fallback to old API
    import mediapipe as mp
    USE_NEW_API = False
    print("✓ Using MediaPipe legacy API")

print("✓ MediaPipe initialized!")

In [ ]:
# Cell 5: Define Helper Functions

def detect_skin_tone(image):
    """
    Simplified skin tone detection using face region color analysis
    Works without MediaPipe face detection
    """
    try:
        # Convert PIL to numpy array
        img_array = np.array(image)
        
        # Use OpenCV's built-in face detection as fallback
        img_cv = cv2.cvtColor(img_array, cv2.COLOR_RGB2BGR)
        gray = cv2.cvtColor(img_cv, cv2.COLOR_BGR2GRAY)
        
        # Load OpenCV's pre-trained face detector
        face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
        faces = face_cascade.detectMultiScale(gray, 1.1, 4)
        
        if len(faces) > 0:
            # Get first face
            x, y, w, h = faces[0]
            
            # Extract face region
            face_region = img_cv[y:y+h, x:x+w]
            
            # Calculate average color
            avg_color = face_region.mean(axis=(0, 1))
            brightness = avg_color.mean()
            
            # Classify skin tone
            if brightness < 100:
                return "deep"
            elif brightness < 150:
                return "medium"
            else:
                return "fair"
        else:
            # No face detected, analyze center region
            h, w = img_array.shape[:2]
            center_region = img_array[h//4:3*h//4, w//4:3*w//4]
            avg_brightness = center_region.mean()
            
            if avg_brightness < 100:
                return "deep"
            elif avg_brightness < 150:
                return "medium"
            else:
                return "fair"
    except Exception as e:
        print(f"Skin tone detection fallback: {e}")
        return "medium"  # Safe default


def cultural_outfit(country, occasion):
    """Determine outfit based on country and occasion"""
    country = country.lower()
    occasion = occasion.lower()
    
    if country == "india":
        if "wedding" in occasion:
            return "traditional indian wedding outfit"
        if "festival" in occasion:
            return "ethnic festive indian wear"
        return "modern indian fusion wear"
    if country == "japan":
        return "modern japanese inspired fashion"
    if country == "france":
        return "elegant parisian fashion"
    if country == "korea":
        return "korean street fashion"
    if country == "middle east":
        return "elegant modest middle eastern fashion"
    
    return "modern stylish outfit"


def budget_style(budget):
    """Determine style based on budget"""
    if budget == "Low":
        return "affordable casual fashion"
    if budget == "Mid":
        return "premium high street fashion"
    return "luxury designer outfit"


print("✓ Helper functions defined!")

In [ ]:
# Cell 6: Main StyleSense Function

def stylesense(
    image, occasion, gender, age_group,
    season, budget, country, user_prompt
):
    """Main function to generate fashion recommendations"""
    
    if image is None:
        return None, "⚠️ Please upload an image to get started!"
    
    # Analyze user's skin tone
    tone = detect_skin_tone(image)
    outfit = cultural_outfit(country, occasion)
    price_style = budget_style(budget)
    
    style_modes = [
        "traditional styling",
        "modern fashion styling",
        "luxury designer styling"
    ]
    
    results = []
    
    print(f"Generating {len(style_modes)} outfit variations...")
    
    for i, mode in enumerate(style_modes, 1):
        print(f"  → Generating look {i}/{len(style_modes)}: {mode}...")
        
        prompt = f"""
        professional fashion photography,
        full body {gender} model,
        age group {age_group},
        
        wearing {outfit},
        {price_style},
        {mode},
        
        season {season},
        suitable for {occasion},
        designed for {tone} skin tone,
        country fashion influence: {country},
        
        {user_prompt},
        
        realistic lighting,
        high detail clothing,
        magazine quality,
        solo model, no extra people
        """
        
        img = pipe(
            prompt,
            negative_prompt="blurry, low quality, extra people, distorted face, multiple people, crowd",
            num_images_per_prompt=1,
            num_inference_steps=40,
            guidance_scale=8
        ).images[0]
        
        results.append(img)
    
    explanation = f"""
🎨 AI Styling Analysis

📊 Your Profile:
• Skin Tone: {tone.capitalize()}
• Age Group: {age_group}
• Gender: {gender}
• Season: {season}
• Budget Level: {budget}
• Cultural Influence: {country}

✨ Generated Looks:
✓ Traditional {country} Style
✓ Modern Fashion Look
✓ Luxury Designer Style

💡 Each look is tailored to your {occasion} occasion!
"""
    
    print("✓ All looks generated successfully!")
    return results, explanation


print("✓ Main StyleSense function ready!")

In [ ]:
# Cell 7: Launch Gradio Interface

interface = gr.Interface(
    fn=stylesense,
    inputs=[
        gr.Image(type="pil", label="📸 Upload Your Photo"),
        gr.Dropdown(
            ["Casual", "Office", "Party", "Wedding", "Festival", "Gym"],
            label="🎯 Occasion",
            value="Casual"
        ),
        gr.Radio(
            ["Male", "Female", "Unisex"],
            label="👤 Gender",
            value="Female"
        ),
        gr.Dropdown(
            ["Teen", "Young Adult", "Adult", "Senior"],
            label="📅 Age Group",
            value="Young Adult"
        ),
        gr.Dropdown(
            ["Summer", "Winter", "Spring", "Fall"],
            label="🌤️ Season",
            value="Summer"
        ),
        gr.Dropdown(
            ["Low", "Mid", "High"],
            label="💰 Budget",
            value="Mid"
        ),
        gr.Dropdown(
            ["India", "USA", "France", "Japan", "Korea", "Middle East"],
            label="🌍 Country Style",
            value="India"
        ),
        gr.Textbox(
            label="✍️ Optional Style Prompt",
            placeholder="e.g., 'add floral patterns' or 'minimalist design'",
            value=""
        )
    ],
    outputs=[
        gr.Gallery(label="👗 Your Personalized Outfit Suggestions", columns=3),
        gr.Textbox(label="📝 AI Styling Explanation", lines=12)
    ],
    title="✨ StyleSense — AI Fashion Recommendation System",
    description="Upload your photo and get personalized fashion recommendations based on your preferences!",
    theme="soft"
)

# Launch with debug mode enabled
interface.launch(share=True, debug=True)

print("\n🚀 StyleSense is now running!")
print("📱 Click the public URL above to access your fashion AI")